In [1]:

import pandas as pd
from datasets import Dataset,DatasetDict
from transformers import AutoTokenizer,AutoModelForMultipleChoice, TrainingArguments, Trainer, BitsAndBytesConfig
from accelerate import Accelerator
import peft
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch
import datetime
import numpy as np


/home/mowgli/miniconda3/envs/textgen2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train = pd.read_csv('train.csv')

In [3]:
examples = pd.read_csv('60k_data/all_12_with_context2.csv')

In [4]:

sentences = [example for example in examples['prompt']]

In [5]:
sentences.extend(train['prompt'].tolist())

In [7]:
len(sentences) - 200

60347

In [6]:
len(sentences)

60547

In [8]:
from sentence_transformers import SentenceTransformer
# sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
# embeddings = model.encode(sentences)
# print(embeddings)

In [9]:
from sentence_transformers import util

In [208]:
# # embeddings = model.encode(sentences, convert_to_tensor=True)

# #Compute cosine-similarities for each sentence with each other sentence
# # cosine_scores = util.cos_sim(embeddings, embeddings)
# paraphrases = util.paraphrase_mining(model, sentences,top_k=20)


# # embed_src = model.encode(train['prompt'][0], convert_to_tensor=True)

In [211]:
# similar_sentences = []

# for paraphrase in paraphrases:
#     score, i, j = paraphrase
#     if (i in range(60347,len(sentences))) and (j < 60347):
#         if .25<= score <= .90:
#             # print("{} \t\t {} \t\t Score: {:.4f}".format(sentences[i], sentences[j], score))
#             similar_sentences.append(sentences[j])


In [214]:
# len(similar_sentences)

281

In [212]:
# examples_2 = pd.read_csv('60k_data/train_with_context2.csv')

In [191]:
# final_similar_questions = pd.concat([examples,examples_2])

In [187]:
# len(list(set(similar_sentences)))

424

In [194]:
# final_similar_questions=final_similar_questions.loc[final_similar_questions['prompt'].isin(list(set(similar_sentences)))]

In [198]:
# final_similar_questions=final_similar_questions.loc[final_similar_questions.duplicated()==False]

In [12]:
sentences=list(set(sentences))

In [13]:
len(sentences)

54409

In [26]:
import os
import csv
import time

model = SentenceTransformer('all-MiniLM-L6-v2')

corpus_embeddings = model.encode(sentences, batch_size=64, show_progress_bar=True, convert_to_tensor=True)


print("Start clustering")
start_time = time.time()

clusters = util.community_detection(corpus_embeddings, min_community_size=10, threshold=0.50)

print("Clustering done after {:.2f} sec".format(time.time() - start_time))

for i, cluster in enumerate(clusters):
    print("\nCluster {}, #{} Elements ".format(i+1, len(cluster)))
    for sentence_id in cluster[0:3]:
        print("\t", sentences[sentence_id])
    print("\t", "...")
    for sentence_id in cluster[-3:]:
        print("\t", sentences[sentence_id])

Batches: 100%|██████████| 851/851 [00:06<00:00, 141.09it/s]


Start clustering
Clustering done after 215.12 sec

Cluster 1, #801 Elements 
	 Which of the following is where are most microbes filtered out of the fluids that accumulate in the body tissues?
	 What is the blood in the body pumped with?
	 About 95 percent of lipids are absorbed in what organ of the digestive system?
	 ...
	 Cell transport refers to the movement of substances across what structure?
	 What is the difference between active and passive transport in cells?
	 What is the body system that digests food mechanically and chemically and absorbs nutrients?

Cluster 2, #750 Elements 
	 What consists of all the biotic and abiotic factors in an area and their interactions?
	 Which condition is the basis for a species to be reproductively isolated from other members?
	 What group of organisms was initially misclassified as plants because they also lack the ability to move, grow in soil, and have cell walls?
	 ...
	 What is the most important process for the survival of a species?
	 I

In [28]:
len(clusters) # 1373 'clusters' / 'communities' based on our parameters above

1373

In [30]:
for id in clusters[1372]:
    print("\t", sentences[id])

	 What is the factor that makes conventional directed motion all but impossible for nanoscale Brownian motors?
	 What are the implications of Brownian motors?
	 What are the finite-dimensional laws of the Brownian tree?
	 Based on the provided information, what phenomenon occurred when the vanes of the paddle wheel interacted with the gas during the Brownian ratchet experiment mentioned in the excerpt?
	 What is the mathematical theory of Brownian motion often referred to as?
	 What is the significance of the Wiener process in the field of probability theory?
	 What is the intuitive property of the Brownian tree?
	 When was the term "Brownian motor" coined?
	 What is the phenomenon behind the theory of Brownian motors?
	 How can the Brownian tree be defined?
